## Coleta de dados estruturados em lista de arquivos

Nesse exercício prático, vamos coletar os dados através de vários arquivos separados por data.
O dataset utilizado é um conjunto de dados ficticios que ilustra vendas de veículos nos 3 úlitmos meses.

Abaixo temos uma amostra de arquivos que contém dados de vendas de veículos por datas.

![image.png](attachment:a23e94e9-5883-4f0e-84c9-9b98b32c28ea.png)

A prática consiste em coletar todos os dados em cada arquivo e adiciona-los em um dataframe do pandas e em seguida analisar e persistir os dados em arquivo de texto e em tabelas no banco de dados.


In [ ]:
#Vamos importar as bibliotecas que vamos utilizar para essa prática
import pandas as pd
import os # bibliotecas para funções do sistema operacional

### Documentação das bibliotecas
**Pandas** - https://pandas.pydata.org/pandas-docs/stable/index.html

**os** - https://docs.python.org/3/library/os.html

Vamos atribuir a uma váriavel o caminho no qual está armazenados os arquivos com as vendas do veículo



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
DIRETORIO = '/content/drive/MyDrive/Colab Notebooks/PÓS CIENT DADOS - XP/MÓDULO 2/vendas_veiculos/'

Agora vamos criar uma lista com todos os arquivos que estão armazenados no diretório.

o metódo listdir retorna uma lista contendo os nomes das entradas no diretório fornecido por path.


In [ ]:
lista_df = [] # cria uma lista
lista_path = os.listdir(DIRETORIO)

In [ ]:
lista_path[:3]

Ótmo! Agora temos todos os nomes dos aquivos que estão dentro do diretório. Agora vamos abrir cada arquivo e coletar os seus dados.

In [ ]:
lista_df = [] # cria uma lista vazia
for arquivo in lista_path:
    local_arquivo = os.path.join(DIRETORIO, arquivo)
    df = pd.read_csv(local_arquivo, sep = ';', encoding='latin-1')
    lista_df.append(df)
df_vendas = pd.concat(lista_df) # Atribui os dados coletados em um dataframe

In [ ]:
df_vendas[:10]

# Exportando dados para arquivos

Agora que ja temos todos os arquivos do diretório, vamos exporta-los apenas para um arquivo único.


In [ ]:
df_vendas.to_csv('/content/drive/MyDrive/Colab Notebooks/PÓS CIENT DADOS - XP/MÓDULO 2/' + 'vendas_por_data.csv', sep = ';', encoding='latin-1', index=False)

Agora temos os dados das vendas em um único arquivo. Abaixo segue a prévia do arquivo exportado.

![image.png](attachment:c3d5cd8c-11c8-46da-94c3-0211cae07ff3.png)

#  Armazendo os dados no banco dados MySQL

Vamos importar as bibliotecas necessárias para trabalhar no banco


In [ ]:
!pip install mysqlclient

In [ ]:
!pip install sqlalchemy

In [ ]:
import MySQLdb # importação da biblioteca mysqlclient
import sqlalchemy

### Documentação das bibliotecas
**MySQLdb** - https://mysqlclient.readthedocs.io/user_guide.html

**sqlalchemy** - https://www.sqlalchemy.org/


### Criando schema do banco de dados
O próximo passo é criar a conexão com o banco de dados. No entanto, antes de executar o script que gera a string de conexão precisamos criar o schema no banco de dados.

Os **schemas** são coleções de objetos dentro de um determinado banco de dados, que organizam vários aspectos e são importantes para segmentação da segurança, facilitando a administração dos objetos e dos dados.

Para criar o o schema de um banco de dados MySQL temos que utilizar o seguinte comando dentro do ambiente do SGDB:

> create schema [nome_schema];

SGBD é o sistema gerenciador de banco de dados. Para esse curso vamos utilizar o MySQL Workbench.

![image.png](attachment:2bd93459-6a91-470b-9c3d-663eb007d07b.png)


Pronto! Depois que criamos o schema/database dentro do MySQL Workbenck o proximo passo é criar a conexão com o banco.
Vamos definir os dados necessárias para fazer o acesso ao servidor de banco de dados MySQL e, em seguinda, criar uma conexão com o banco de dados baseado em uma string de conexão.

In [ ]:
user = 'leandroigti'
password = 'Igti123456'
host = 'localhost'
database = 'coletaIGTI' #schema

string_conexao = f'mysql://{user}:{password}@{host}/{database}'
string_conexao #testar a string de conexão com o banco de dados.

A string é composta pelo o **usuário** de acesso ao banco, a **senha**, o **caminho** do banco de dados e o **schema** que vai ser utilizado.

In [ ]:
engine = sqlalchemy.create_engine(string_conexao)
conn = engine.connect()

Conexão realizada! Agora vamos voltar no MySQL Workbench e criar as tabelas que vamos inserir os dados coletados.
No nosso dataset podemos identificar informações de **pessoa, veiculo, fabricante e venda**. Desta forma, vamos criar uma tabela no banco de dados para cada assunto.

Abaixo segue o diagrama de entidade e relacionamento modelado.

![image.png](attachment:7528338d-0da5-4b35-842f-ae04ccff561c.png)

Criado o modelo de entidade e relacionamento, o proximo passo é preparar os dados para inserir nas tabelas.

Inicialmente vamos trabalhar os dados que **não** possuem dependecias de outras tabelas.

In [ ]:
df_pessoas = df_vendas[['cod_pessoa', 'nome', 'genero', 'data_nascimento']].drop_duplicates()
df_modelos = df_vendas[['modelo', 'fabricante']].drop_duplicates()
fabricantes = list(df_vendas.fabricante.unique())

Observerm que para o fabricanes não foi gerado um dataframe e sim uma lista. Não foi necessário criar um dataframe, uma vez que os dados contidos são apenas de uma coluna.

In [ ]:
df_pessoas[:10] #dados de pessoas

In [ ]:
df_modelos #Dados de modelo

In [ ]:
fabricantes # dados de fabricante

O primeiro passo é inserir os dados do dataframe **df_pessoas** na tabela no banco.
Para isso, vamos percorrer todo o dataframe e atribuir as variáveis os valores de cada coluna. Em seguida, vamos inseri-las no insert.

In [ ]:
for pessoa in df_pessoas[:3].itertuples():
    print(pessoa)

In [ ]:
for pessoa in df_pessoas.itertuples():
    cod_pessoa = pessoa.cod_pessoa
    nome = pessoa.nome
    genero = pessoa.genero
    data_nascimento = str(pessoa.data_nascimento)
    try:
        query = f"""insert into pessoa (cod_pessoa, nome, genero, data_nascimento )
                     values ('{cod_pessoa}', '{nome}',  '{genero}', '{data_nascimento}')"""
        conn.execute(query)
        print(f"Registro inserido com sucesso: {nome}")
    except Exception as e:
        print(f"Não foi possível inserir o regristro {nome}. O erro foi encontrado foi: {e}")

Da mesma forma realizada no dataframe de pessoas, vamos fazer na lista de fabricantes.

In [ ]:
for fabricante in fabricantes:
    try:
        query = f"""insert into fabricante (nome_fabricante)
                     values ('{fabricante}')"""
        conn.execute(query)
        print(f"Registro inserido com sucesso: {fabricante}")
    except Exception as e:
        print(f"Não foi possível inserir o regristro {fabricante}. O erro foi encontrado foi: {e}")

Pronto! Ja inserimos os registros de fabricante na tabela no banco. Agora vamos ler um sql da tabela do banco e adiciona-lo em um dataframe.

In [ ]:
df_fabricante = pd.read_sql('fabricante', con=conn) # Lê o dado na tabela do banco de dados

In [ ]:
df_fabricante

Perceba que agora temos o código associado a um fabricante. Esse código será utilizado para popular a tabela de modelo_veiculo.

![image.png](attachment:54f7a90a-4336-4c88-a180-042e9a5e12ef.png)

Perceba que na tabela de modelo_veiculo possui uma dependencia da tabela fabricante. Essa dependência esta relacionada ao código do fabricante. Dessa forma, temos que buscar esse código na tabela de Fabricante e inseri-lo na tabela de modelo_veiculo.

![image.png](attachment:f5c9e441-bfaa-4de5-90c4-1306c188432d.png)

Para fazer isso vamos utilizar o dataframe df_fabricante criado a partir do insert no banco e o dataframe df_modelos. Então o que fazer?
> *Vamos ler todo o dado contido na coluna de fabricante do dataframe **df_modelos** e buscar a sua equivalencia no dataframe **df_fabricante***



 Primeiro vamos setar a coluna nome de fabricante como index. Essa coluna vai ser utilizada como index para buscar os dados no dataframe df_fabricante. Após isso, vamos transformar em um dicionário

In [ ]:

dict_fabricante = df_fabricante.set_index(df_fabricante.nome_fabricante)['cod_fabricante'].to_dict()
dict_fabricante


Nesse momento vamos buscaros nomes dos fabricantes e associamos a seu respectivo código.

In [ ]:
df_modelos.fabricante = df_modelos.fabricante.map(dict_fabricante)
df_modelos


Feito isso, vamos alterar o nome da coluna.

In [ ]:
df_modelos = df_modelos.rename(columns={'fabricante' : 'cod_fabricante'})
df_modelos


Pronto! Agora é so fazer o insert na tabela do banco.

In [ ]:
for modelo in df_modelos.itertuples():
    nome_modelo = modelo.modelo
    cod_fabricante = modelo.cod_fabricante
    try:
        query = f"""insert into modelo_veiculo (descricao_modelo, cod_fabricante)
                     values ('{nome_modelo}',{cod_fabricante})"""
        conn.execute(query)
        print(f"Registro inserido com sucesso: {nome_modelo}")
    except Exception as e:
        print(f'Não foi possível inserir o registro: {nome_modelo} Erro: {e}')


## Criando tabela Staging no banco de dados

Uma outra forma de popular dados no banco de dados é através da criação de uma tabela de preparação temporária. Uma tabela Staging é uma area intermediária utilizada para preparar os dados para serem utilizados em um ambiente de análise.

Deste modo, criamos uma cópia da tabela com os dados coletados e persistimos no banco sem nenhum tratamento.

In [ ]:
# cria uma tabela no banco SQL com o nome
df_vendas.to_sql('stg_venda_veiculos', con=conn, schema='coletaIGTI')

Após criar a tabela de staging no banco de dados, vamos executar a seguinte consulta em SQL abaixo:

![image.png](attachment:3006f2f5-7863-4398-a336-74c2837947fa.png)

Na tabela principal **stg_venda_veiculos** vamos selecionar os campos: data_compra, valor e cod_pessoa. Além disso, temos que buscar a informação de cod_modelo que esta na tabela **modelo_veiculo**. Para fazer isso vamos realizar uma junção entre as duas tabelas e indicar o campo de descrição como parâmetro. Feito isso, vamos ter os dados necessários para inserir na tabela **venda_veiculo**.

Pronto! Agora temos todos os dados coletados e armazenados normalizados no banco de dados. \0/